In [4]:
import spacy
import pdfminer
import re
import os
import pandas as pd
import pdf2txt

In [5]:
def convert_pdf(f):
    output_filename = os.path.basename(os.path.splitext(f)[0]) + ".txt" # removing pdf extension and appending txt
    output_filepath = os.path.join("output/txt.", output_filename) # file txt will be saved in this folder txt
    pdf2txt.main(args=[f, "--outfile", output_filepath]) # pdf to txt and save it in given location
    print(output_filepath + "saved successfully!!!")
    return open(output_filepath).read() # open and read the file

In [6]:
# Load the language model
nlp = spacy.load("en_core_web_sm")

In [ ]:
# phone_num credit https://stackoverflow.com/a/3868861

In [8]:
# Create the output file structure
result_dict = {'name': [], 'phone': [], 'email': [], 'skills': []}
names = []
phones = []
emails = []
skills = []

In [14]:
def parse_content(text):
    skillset = re.compile("python|java|sql|hadoop|tableau")
    phone_num = re.compile(
        "(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})"
    )
    doc = nlp(text)
    name = [entity.text for entity in doc.ents if entity.label_ is "PERSON"][0] # first name detected
    print(name)
    email = [word for word in doc if word.like_email == True][0] # give the first email id
    print(email)
    phone = str(re.findall(phone_num, text.lower()))
    skills_list = re.findall(skillset, text.lower())
    unique_skills_list = str(set(skills_list))
    
    names.append(name)
    emails.append(email)
    phones.append(phone)
    skills.append(unique_skills_list)
    print("Extraction completed successfully!!!")

<>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\Dragan Golic\AppData\Local\Temp\ipykernel_15260\776383585.py:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
  name = [entity.text for entity in doc.ents if entity.label_ is "PERSON"][0]


In [15]:
# Parse more resumes at the time
for file in os.listdir('resumes/'):
    if file.endswith('.pdf'):
        print('Reading.....' + file)
        txt = convert_pdf(os.path.join('resumes/',file))
        parse_content(txt)

Reading.....Alisson ParkerCV.pdf
output/txt.\Alisson ParkerCV.txtsaved successfully!!!
Alisson Parker-Wright                                                                                                 
alli1414parks@mail.com
Extraction completed successfully!!!
Reading.....AshleyMilesCV.pdf
output/txt.\AshleyMilesCV.txtsaved successfully!!!
Ashley Miles
ashleymiles@memail.com
Extraction completed successfully!!!
Reading.....John DominicCV.pdf
output/txt.\John DominicCV.txtsaved successfully!!!
John Dominic
johndominic@mail.com
Extraction completed successfully!!!


In [17]:
names

['Alisson Parker-Wright                                                                                                 ',
 'Ashley Miles',
 'John Dominic']

In [18]:
phones

["['8569878511']", "['6592251422']", "['7877756411']"]

In [19]:
skills

["{'java', 'tableau', 'python'}",
 "{'sql', 'tableau'}",
 "{'java', 'hadoop', 'python'}"]

In [16]:
result_dict['name'] = names
result_dict['phone'] = phones
result_dict['email'] = emails
result_dict['skills'] = skills
print(result_dict)

{'name': ['Alisson Parker-Wright                                                                                                 ', 'Ashley Miles', 'John Dominic'], 'phone': ["['8569878511']", "['6592251422']", "['7877756411']"], 'email': [alli1414parks@mail.com, ashleymiles@memail.com, johndominic@mail.com], 'skills': ["{'java', 'tableau', 'python'}", "{'sql', 'tableau'}", "{'java', 'hadoop', 'python'}"]}


In [21]:
result_df = pd.DataFrame(result_dict)
result_df

,name,phone,email,skills
0,Alisson Parker-Wright ...,['8569878511'],alli1414parks@mail.com,"{'java', 'tableau', 'python'}"
1,Ashley Miles,['6592251422'],ashleymiles@memail.com,"{'sql', 'tableau'}"
2,John Dominic,['7877756411'],johndominic@mail.com,"{'java', 'hadoop', 'python'}"


In [22]:
result_df.to_csv('output/csv/parsed_resumes.csv')